In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
from reservoirpy.nodes import Reservoir, Ridge, ESN
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import pickle
import matplotlib.pyplot as plt
import datetime

In [24]:
# Configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 64
EMBEDDING_SIZE = 256  # Taille des embeddings pour tous les inputs
NUM_HEADS = 8
NUM_LAYERS = 4
DROPOUT = 0.1
LEARNING_RATE = 1e-4
EPOCHS = 20
PATIENCE = 5

In [25]:
class MultiTaskTransformer(nn.Module):
    def __init__(
        self, input_size, embedding_size, num_heads, num_layers, num_labels, dropout
    ):
        super(MultiTaskTransformer, self).__init__()
        self.embedding = nn.Linear(input_size, embedding_size)  # Embedding Layer
        self.transformer = nn.Transformer(
            d_model=embedding_size,
            nhead=num_heads,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dropout=dropout,
            batch_first=True,
        )
        self.classifier = nn.Linear(embedding_size, num_labels)  # Final Classifier

    def forward(self, x):
        # Pass input through the embedding layer
        embedded = self.embedding(x)

        # Add a positional encoding (if needed)
        embedded = embedded.unsqueeze(1)  # Add sequence dimension

        # Transformer expects (batch, seq_len, embedding_size)
        transformer_output = self.transformer(embedded, embedded)

        # Take only the output of the first token (classification token equivalent)
        output = transformer_output[:, 0, :]  # Extract first token

        # Pass through the classifier
        predictions = self.classifier(output)
        return predictions


In [26]:
def load_data():
    partOfData = 1
    X_genres = pd.read_csv("../data/test/input_genres_tags_data.csv")
    X_instruments = pd.read_csv("../data/test/input_instruments_tags_data.csv")
    X_moods = pd.read_csv("../data/test/input_moods_tags_data.csv")

    y_genres = pd.read_csv("../data/test/output_genres_tags_data.csv")
    y_instruments = pd.read_csv("../data/test/output_instruments_tags_data.csv")
    y_moods = pd.read_csv("../data/test/output_moods_tags_data.csv")

    # On peut garder seulement une partie des données
    X_genres = X_genres[: int(partOfData * len(X_genres))]
    X_instruments = X_instruments[: int(partOfData * len(X_instruments))]
    X_moods = X_moods[: int(partOfData * len(X_moods))]
    y_genres = y_genres[: int(partOfData * len(y_genres))]
    y_instruments = y_instruments[: int(partOfData * len(y_instruments))]
    y_moods = y_moods[: int(partOfData * len(y_moods))]

    return (X_genres, X_instruments, X_moods), (y_genres, y_instruments, y_moods)


# Ensure the input data is in the correct format
def reshape_input(X):
    if isinstance(X, pd.DataFrame):
        return X.values.reshape(-1, 1, X.shape[1])  # Handles pandas DataFrame
    elif isinstance(X, np.ndarray):
        return X.reshape(-1, 1, X.shape[1])  # Handles numpy ndarray
    else:
        raise ValueError("Input must be a pandas DataFrame or a numpy ndarray")


def format_predictions(predictions):
    # Convert the list to a NumPy array
    predictions_array = np.array(predictions)

    # Reshape the array to 2-dimensional
    predictions_reshaped = predictions_array.reshape(-1, predictions_array.shape[-1])

    return predictions_reshaped

In [27]:
# Charger les données
(X_genres, X_instruments, X_moods), (y_genres, y_instruments, y_moods) = load_data()

# Préparation des données
X_genres = X_genres.drop(columns=["ChallengeID"])
X_instruments = X_instruments.drop(columns=["ChallengeID"])
X_moods = X_moods.drop(columns=["ChallengeID"])
y_genres = y_genres.drop(columns=["ChallengeID"])
y_instruments = y_instruments.drop(columns=["ChallengeID"])
y_moods = y_moods.drop(columns=["ChallengeID"])


X = np.concatenate([X_genres, X_instruments, X_moods], axis=1)

y = np.concatenate([y_genres, y_instruments, y_moods], axis=1)

# Convertir les données en tensors PyTorch
X_tensor = torch.tensor(X, dtype=torch.float32).to(DEVICE)

y_tensor = torch.tensor(y, dtype=torch.float32).to(DEVICE)

# Générer les données croisées pour les interactions
X_genres_instruments = np.concatenate([X_genres, X_instruments], axis=1)
X_genres_moods = np.concatenate([X_genres, X_moods], axis=1)
X_instruments_moods = np.concatenate([X_instruments, X_moods], axis=1)
y_genres_instruments = np.concatenate([y_genres, y_instruments], axis=1)
y_genres_moods = np.concatenate([y_genres, y_moods], axis=1)
y_instruments_moods = np.concatenate([y_instruments, y_moods], axis=1)

In [28]:
# Load ENSs models
sub_folder = 0
while os.path.exists(f"../models/model_{sub_folder}"):
    sub_folder += 1
sub_folder -= 1

print(f"Loading ESNs models from ../models/model_{sub_folder}")

with open(f"../models/model_{sub_folder}/esn_Genre.pkl", "rb") as f:
    model_Genre = pickle.load(f)

with open(f"../models/model_{sub_folder}/esn_Instrument.pkl", "rb") as f:
    model_Instrument = pickle.load(f)

with open(f"../models/model_{sub_folder}/esn_Mood.pkl", "rb") as f:
    model_Mood = pickle.load(f)

with open(f"../models/model_{sub_folder}/esn_Genre_Instrument.pkl", "rb") as f:
    model_Genre_Instrument = pickle.load(f)

with open(f"../models/model_{sub_folder}/esn_Genre_Mood.pkl", "rb") as f:
    model_Genre_Mood = pickle.load(f)

with open(f"../models/model_{sub_folder}/esn_Instrument_Mood.pkl", "rb") as f:
    model_Instrument_Mood = pickle.load(f)

# Load Transformer model
with open(f"../models/model_{sub_folder}/transformer.pkl", "rb") as f:
    model_Transformer = pickle.load(f)

# Affichage du nombre de paramètres
print(
    f"Nombre total de paramètres : {sum(p.numel() for p in model_Transformer.parameters())}"
)

Loading ESNs models from ../models/model_2
Nombre total de paramètres : 11830776


In [29]:
X_genres_reshaped = reshape_input(X_genres)
X_instruments_reshaped = reshape_input(X_instruments)
X_moods_reshaped = reshape_input(X_moods)

y_genres_reshaped = reshape_input(y_genres)
y_instruments_reshaped = reshape_input(y_instruments)
y_moods_reshaped = reshape_input(y_moods)

# Reshape les données croisées pour les ESNs
X_genres_instruments_reshaped = reshape_input(X_genres_instruments)
X_genres_moods_reshaped = reshape_input(X_genres_moods)
X_instruments_moods_reshaped = reshape_input(X_instruments_moods)

y_genres_instruments_reshaped = reshape_input(y_genres_instruments)
y_genres_moods_reshaped = reshape_input(y_genres_moods)
y_instruments_moods_reshaped = reshape_input(y_instruments_moods)

In [30]:
# Obtenir les sorties des réservoirs
y_genres_pred = model_Genre.run(X_genres_reshaped)
y_instruments_pred = model_Instrument.run(X_instruments_reshaped)
y_moods_pred = model_Mood.run(X_moods_reshaped)

# Formater les prédictions
y_genres_pred = format_predictions(y_genres_pred)
y_instruments_pred = format_predictions(y_instruments_pred)
y_moods_pred = format_predictions(y_moods_pred)

# Obtenir les sorties des réservoirs croisés
y_genres_instruments_pred = model_Genre_Instrument.run(X_genres_instruments_reshaped)
y_genres_moods_pred = model_Genre_Mood.run(X_genres_moods_reshaped)
y_instruments_moods_pred = model_Instrument_Mood.run(X_instruments_moods_reshaped)


# Formater les prédictions
y_genres_instruments_pred = format_predictions(y_genres_instruments_pred)
y_genres_moods_pred = format_predictions(y_genres_moods_pred)
y_instruments_moods_pred = format_predictions(y_instruments_moods_pred)


# Combine toutes les sorties (individuelles et croisées)
X_reservoirs = np.concatenate(
    [
        y_genres_pred,
        y_instruments_pred,
        y_moods_pred,
        y_genres_instruments_pred,
        y_genres_moods_pred,
        y_instruments_moods_pred,
    ],
    axis=1,
)

Running ESN-20: 100%|██████████| 47615/47615 [00:01<00:00, 36987.01it/s]


In [31]:
# Création des datasets
main_dataset = torch.utils.data.TensorDataset(
    torch.tensor(X_reservoirs, dtype=torch.float32).to(DEVICE),
    y_tensor.clone().detach().to(DEVICE),
)

# Création des loaders
main_loader = torch.utils.data.DataLoader(
    main_dataset, batch_size=BATCH_SIZE, shuffle=True
)

In [38]:
# Evaluation des performances (accuracy, precision, recall, f1-score)
def evaluate_performance(model, test_loader):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            predictions = model(X_batch)
            predictions = torch.sigmoid(predictions)
            predictions = (predictions > 0.5).int()
            y_true.append(y_batch.cpu().numpy())
            y_pred.append(predictions.cpu().numpy())
    y_true = np.concatenate(y_true, axis=0)
    y_pred = np.concatenate(y_pred, axis=0)

    # Save 5% of the rows of the predictions as csv files in the data folder in predictions folder with timestamp
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

    np.savetxt(
        f"../data/predictions/y_true_{timestamp}.csv",
        y_true[: int(0.05 * len(y_true))],
        delimiter=",",
    )
    np.savetxt(
        f"../data/predictions/y_pred_{timestamp}.csv",
        y_pred[: int(0.05 * len(y_pred))],
        delimiter=",",
    )
    
    # Histograms plot of the predictions and true values for each tag
    # for i in range(y_true.shape[1]):
    #     plt.hist(y_true[:, i], bins=2, alpha=0.5, label="True")
    #     plt.hist(y_pred[:, i], bins=2, alpha=0.5, label="Predicted")
    #     plt.title(f"Tag {i}")
    #     plt.legend()
    #     plt.savefig(f"../data/predictions/histogram_tag_{i}.png")
    #     plt.clf()

    # Accuracy
    accuracy = np.mean(y_true == y_pred)
    print(f"Accuracy: {accuracy}")

    # Precision, Recall, F1-Score
    from sklearn.metrics import classification_report

    report = classification_report(y_true, y_pred, digits=4)
    print(report)


In [39]:
evaluate_performance(model_Transformer, main_loader)

Accuracy: 0.9901951302957526
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000         2
           1     0.6732    0.7891    0.7265      1261
           2     0.0000    0.0000    0.0000         0
           3     0.0000    0.0000    0.0000         0
           4     0.2439    0.9091    0.3846        11
           5     0.1166    0.8852    0.2061        61
           6     0.1622    0.9231    0.2759        13
           7     0.1055    0.7073    0.1835        41
           8     0.4561    0.7391    0.5641       506
           9     0.1223    0.9583    0.2170        24
          10     0.7522    0.9767    0.8498      1243
          11     0.8600    0.8521    0.8561      4226
          12     0.3720    0.9045    0.5272       220
          13     0.1412    0.8404    0.2418       188
          14     0.1492    0.7833    0.2507       120
          15     0.1768    0.9667    0.2990        60
          16     0.0000    0.0000    0.0000         

/home/elouarn/Documents/Projet_IA/challenge-data-music-catalogs/music-catalogs-classifier/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/elouarn/Documents/Projet_IA/challenge-data-music-catalogs/music-catalogs-classifier/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/elouarn/Documents/Projet_IA/challenge-data-music-catalogs/music-catalogs-classifier/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill